# Abstract

This notebook approaches the uses a toy dataset, the [Google Quick Draw! dataset](https://github.com/googlecreativelab/quickdraw-dataset#preprocessed-dataset) to peform initial experiments iwth Generative Adversarial Network learning. The book calls these "Ganimals" but this has a specific meaning elsewhere.

# Prelude

## Imports

In [7]:
from typing import NamedTuple, List, Union, Tuple, Optional, Dict

import numpy as np
import numpy.random as rd
import sys
import math
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from tensorflow.keras.layers import Flatten, Dense, Input, Conv2D, Conv2DTranspose, \
  Activation, LeakyReLU, Dropout, BatchNormalization, Reshape, Lambda, UpSampling2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop

from tensorflow.keras import backend as K  # tensor operations

In [8]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logging.info("Logging set up")

08:45:22 INFO:Logging set up


## Configuration

In [9]:
RUNNING_ON_COLAB = False
try:
    from google.colab import drive
    RUNNING_ON_COLAB = TRUE
except:
    logging.info("Running locally. Not running on Google Colab")

08:45:23 INFO:Running locally. Not running on Google Colab


In [10]:
CHECKPOINT_DIR = pathlib.Path.cwd()  / 'checkpoints' / 'celeb_faces'

if not CHECKPOINT_DIR.exists():
  CHECKPOINT_DIR.mkdir(parents=True)

In [11]:
USER_FIRST_NAME: Optional[str] = "bryan"

if RUNNING_ON_COLAB:
    GDRIVE_DIR_BASE = pathlib.Path('/') / 'content' / 'drive'
    MY_GDRIVE_DIR = GDRIVE_DIR_BASE / 'MyDrive'
else:
    # Assume we're running my laptop in Windows
    MY_GDRIVE_DIR = pathlib.Path.home() / 'Google Drive'
    if not MY_GDRIVE_DIR.exists():
        # Assume we're running my laptop in Linux, with GDrive on the Windows partition
        ROOT_DIR = pathlib.Path(pathlib.Path.home().root)
        WIN_DRIVE = ROOT_DIR / 'media' / pathlib.Path.home().name / 'Blade 15'
        WIN_HOME = WIN_DRIVE / 'Users' / pathlib.Path.home().name
        if not WIN_HOME.exists():
            import os
            import pwd
            first_name = USER_FIRST_NAME or pwd.getpwuid(os.getuid())[4].lower().split(' ')[0]
            WIN_HOME = WIN_DRIVE / 'Users' / first_name

        MY_GDRIVE_DIR = WIN_HOME / 'Google Drive'
        
COLAB_DATA_DIR = MY_GDRIVE_DIR / 'Colab Data'
CAMEL_NPY_FILE = COLAB_DATA_DIR / 'Google Quick Draw' / 'full_numpy_bitmap_camel.npy' 
assert CAMEL_NPY_FILE.exists()

In [12]:
logging.basicConfig()
level = logging.INFO
logger = logging.getLogger()
logger.setLevel(level)
for handler in logger.handlers:
    handler.setLevel(level)

## Constants

In [13]:
QUICK_DRAW_SHAPE = (28, 28, 1)

## Check GPU Enabled

In [14]:
# Tensorflow v2.4 on CUDA 11

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)  

# Can also write
# import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [15]:
from tensorflow.python.client import device_lib

local_devices = device_lib.list_local_devices()
print(local_devices)

assert "GPU" in [d.device_type for d in local_devices], "No GPU Enabled"

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10356340475750710756
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7601975456
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15091939494646035303
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Super with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


# Data Exploration

In [16]:
camels = np.load(CAMEL_NPY_FILE)
CAMEL_NPY_FILE

PosixPath('/media/bfeeney/Blade 15/Users/bryan/Google Drive/Colab Data/Google Quick Draw/full_numpy_bitmap_camel.npy')

In [17]:
def plot_camels(camels: np.ndarray) -> None:
    figscale = 8
    ncols = 8
    
    if camels.shape[0] > 64:
        figscale /= 4
        ncols *= 4
        
    nrows = min(32, max(1, int(camels.shape[0] / ncols)))

    fig, axes = plt.subplots(figsize=(figscale*ncols, figscale*nrows), dpi=120, nrows=nrows, ncols=ncols)

    row = -1
    for i in range(nrows * ncols):
        row = i // ncols
        col = i % ncols
        ax = axes[row, col]

        ax.imshow(camels[i].reshape(QUICK_DRAW_SHAPE), cmap='Greys')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        
    fig.tight_layout()
    
plot_camels(camels)

So learning to draw a camel from this dataset would be an astonishing feat. For one thing, they're facing different directions: most left, but some right.

# Network Setup

So a GAN consists of two competing components

1. A discriminator which, given a bunch of pixels, states whether it is a real image or a fake. It's trained (at least partly?!) on real image
2. A generator, which generates pixels that confuse the discriminator.
    
In a way it's sort of a reinforcement-learning setup, where you learn the... value function?

There are two ways of expanding out from a latent to the true dimension.

1. Previously we looked at Conv2DTranspose, which inserts zero values inbetween strides
    1. An issue with this is that even when results are good, you can see a checkerboard pattern in the find detail.
2. However an alternative is to Upsample2D (which just repeats values to fill in the gaps) and then use a _standard_ convolution
    1. How the fuck _this_ works is open for debate but *shrug*
    
 Things not mentioned in the book
 
  1. The LeakyReLU units have `alpha=0.2`. The default is 0.3. What is `alpha` is described below
  2. The weight init is N(0, 0.2), and not one of the more complex Glorot ones.
  
 LeakyReLU definition is
 
 $$
 \begin{aligned}
 f(x) & = \left\{  \begin{array}{lr}
     \alpha \cdot x & x < 0 \\
     x & x \geq 0
 \end{array}\right.
 &
 \implies f'(x) & = \left\{  \begin{array}{lr}
     \alpha & x < 0 \\
     1 & x \geq 0
 \end{array}\right.
 \end{aligned}
 $$
 
 Essentially $\alpha$ is the negative slope coefficient. sUually, one chooses $0 < \alpha < 1$. The special case of $\alpha=0$ is an ordinary ReLU, and the special case of $\alpha=1$ is just the identity function. Choosing $\alpha > 1$ implies that the composition of many such layers might exhibit exploding gradients, which is undesirable. Also, choosing $\alpha<0$ makes $f$ a non-monotonic function shaped something like a V. Non-monotonic functions have recently become more popular (e.g. mish and swish)

In [ ]:
class GanBuilder(NamedTuple):
    input_dim: Tuple[int, int, int]
    
    disc_conv_filter_count: List[int]
    disc_conv_kernel_size: List[int]
    disc_conv_strides: List[int]
    disc_batch_norm_momentum: Optional[float]
    disc_activation: str
    disc_dropout_rate: Optional[float]
    disc_learning_rate: float
    
    gen_initial_dense_layer_size: Tuple[int, int, int]
    gen_upsample: List[int]
        
    gen_conv_filter_count: List[int]
    gen_conv_kernel_size: List[int]
    gen_conv_strides: List[int]
    gen_batch_norm_momentum: Optional[float]
    gen_activation: str
    gen_dropout_rate: Optional[float]
    gen_learning_rate: float
        
    optimiser: str
    
    z_dim: int
        
    weight_init: str = 'glorot_uniform'  # they use keras.initializers.RandomNormal(mean=0, stddev=0.02)
        
    models: Dict[str, Optional[Model]] = {'d': None, 'g': None, 'c': None}
        
    disc_loss_real: List[float] = []
    disc_loss_fake: List[float] = []
    disc_acc_real: List[float] = []
    disc_acc_fake: List[float] = []
    gen_loss: List[float] = []
    gen_acc: List[float] = []
        
    epochs_completed_wrapper: List[int] = [ 0 ]  # Hack around immutability of namedtuple
        
    
    @property
    def epochs_completed(self) -> int:
        return self.epochs_completed_wrapper[0]
    
    @epochs_completed.setter
    def epochs_completed(self, val: int) -> None:
        self.epochs_completed_wrapper[0] = val
        
    @property
    def disc_loss_avg(self) -> List[float]:
        return list(0.5 * np.array(self.disc_loss_real) + np.array(self.disc_loss_fake))
    
    @property
    def disc_acc_avg(self) -> List[float]:
        return list(0.5 * np.array(self.disc_acc_real) + np.array(self.disc_acc_fake))
        
    @property
    def discriminator_model(self) -> Model:
        if not self.models.get('d', None):
            self.models['d'] = self._build_discriminator()
        return self.models['d']
        
    @property
    def generator_model(self) -> Model:
        if not self.models.get('g', None):
            self.models['g'] = self._build_generator()
        return self.models['g']
        
    @property
    def compiled_model(self) -> Model:
        """
        Returns a trainable no-op model, that takes a latent representation
        of an image as the input, generates an image from it, and feeds it
        to the discriminator, finally outputing a real / fake classification.
        
        This permutes the discriminator model: it disables training.
        """
        assert self.disc_learning_rate > self.gen_learning_rate, \
            "Want a 'stronger' discriminator than generator, so prefer to make larger, faster steps to convergence"
        
        if not self.models.get('m', None):
            # Compile the discriminator model, and freeze it
            self.discriminator_model.compile(
                optimizer=RMSprop(lr=self.disc_learning_rate),
                loss='binary_crossentropy',  # it's a classification problem
                metrics=['accuracy']
            )

            # Use the discriminator model to create and input / output pair
            # and then use it and that pair to build a paired model
            self.discriminator_model.trainable = False
            
            model_input = Input(shape=(self.z_dim, ), name='model_input')
            model_output = self.discriminator_model(
                self.generator_model(
                    model_input
                )
            )
            model = Model(model_input, model_output)
            
            model.compile(
                optimizer=RMSprop(lr=self.gen_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy']
            )
            
            self.models['m'] = model
            self.discriminator_model.trainable = True
            
        return self.models['m']
            
    
    def train(self, x_train: np.ndarray, epoch_count: int = 2000, batch_size: int = 64) -> Tuple[Model, Model]:
        """
        Since this is a GAN, we have two models to train in an alternating fashion
        
        First we train the discriminator on a binary classification problem using real images
        (labelled 1) and fake images (labelled 2)
        
        Then we fix the discriminator, and train a paired model of generator and discriminator,
        on a binary classification of low-dimensional representations, all of which are labelled
        1 (real). Since only the generator parameters are trainable, the solution to this 
        optimisation problem is to make the generator generate the most plausible images according
        to the discriminator.
        """
        # TODO xtrain should be Union[np.ndarray, ImageDataGenerator] in which either xtrain=xtrain[idx]
        # or xtrain=next(xtrain) with an assert on batch_sizes
        
        _ = self.compiled_model  # Make sure the models are all compiled
        
        for epoch in range(self.epochs_completed, self.epochs_completed + epoch_count):
            self.discriminator_model.trainable = True
            for _ in range(10):
                self._train_discriminator(x_train, batch_size)
            
            self.discriminator_model.trainable = False
            self._train_generator(batch_size)
            self.log_last_epoch_stats_to_info(epoch)
            self.epochs_completed += 1
            
        
    def log_last_epoch_stats_to_info(self, epoch) -> None:
        msg = f"{epoch:04d} [D loss: ({self.disc_loss_avg[-1]:.3f})(R {self.disc_loss_real[-1]:.3f}, F {self.disc_loss_fake[-1]:.3f})] " \
              f"[D acc: ({self.disc_acc_avg[-1]:.3f})(R {self.disc_acc_real[-1]:.3f}, F {self.disc_acc_fake[-1]:.3f})] " \
              f"[G loss: {self.gen_loss[-1]:.3f}] [G acc: {self.gen_acc[-1]:.3f}]"
        
        logging.info(msg)
        
        
    def _low_dim_image_sample(self, batch_size: int) -> np.ndarray:
        # FIXME Generator doesn't adhere to stdev of real generator... in fact using a std normal seems dodge as fuck...,
        #       Shouldn't it be MV Beta?
        fake_image_low_dim = rd.normal(0, 1, (batch_size, self.z_dim))
        return fake_image_low_dim
    
        
    def _train_discriminator(self, x_train: np.ndarray, batch_size: int) -> None:
        """
        This trains just the discriminator on its own, on a simple binary classification problem,
        where real images are labelled as 1, and images from the generator are labelled as 0.
        """
        assert self.discriminator_model.trainable, "Discriminator model is not trainable"
        assert all(l.trainable for l in self.discriminator_model.layers)
        
        # Train on a `batch_size` subset (selected with replacement) of the real images
        batch_indices = rd.randint(0, x_train.shape[0], batch_size)
        real_images_batch = x_train[batch_indices]
        l, a = self.discriminator_model.train_on_batch(real_images_batch, np.ones((batch_size, 1)))
        self.disc_loss_real.append(l)
        self.disc_acc_real.append(a)
        
        # Train on a `batch_size` sample (in practice selected without replacement) of fake generated images
        fake_image_low_dim = self._low_dim_image_sample(batch_size)
        fake_images_batch = self.generator_model.predict(fake_image_low_dim)
        l, a = self.discriminator_model.train_on_batch(fake_images_batch, np.zeros((batch_size, 1)))
        self.disc_loss_fake.append(l)
        self.disc_acc_fake.append(a)
    
    def _train_generator(self, batch_size: int) -> None:
        """
        This _implicitly_ trains the generator, by training the entire generator-discriminator 
        model-pair. The problem is binary classification, where the input to the model is an
        arbitrary low-dim representation, and the output is 1 (i.e. real). 
        
        Since the discriminator should be frozen, the only trainable parameters are those in the
        generator, so the only solution to the problem is to make the generator's images look
        real, by updating the parameters of the generator.
        """
        assert not self.discriminator_model.trainable, \
            "Discriminator model should be frozen (i.e. not trainable) so we update only the generator"
        assert all(not l.trainable for l in self.discriminator_model.layers)
        
        fake_image_low_dim = self._low_dim_image_sample(batch_size)
        l, a = self.compiled_model.train_on_batch(fake_image_low_dim, np.ones((batch_size, 1)))
        self.gen_loss.append(l)
        self.gen_acc.append(a)
        
    
    def _build_discriminator(self) -> Model:
        def n(part: str, idx: int) -> str:
            return f"disc_{part}_{idx:02}"
        
        input_layer = Input(shape=self.input_dim, name="disc_input")
        x = input_layer
        for idx, (fltrs, ksize, stride) in enumerate(zip(self.disc_conv_filter_count,
                                                         self.disc_conv_kernel_size,
                                                         self.disc_conv_strides)
                                                    ):
            idx += 1  # One indexed
            x = Conv2D(filters=fltrs,
                       kernel_size=ksize,
                       strides=stride,
                       padding="same",
                       name=n('conv', idx))(x)
            if self.disc_batch_norm_momentum:
                x = BatchNormalization(momentum=self.disc_batch_norm_momentum,
                                       name=n('bnorm', idx))(x)
            x = Activation(self.disc_activation,
                           name=n(self.disc_activation, idx)
                          )(x)
            if self.disc_dropout_rate:
                x = Dropout(self.disc_dropout_rate, name=n('dropout', idx))(x)
            
        x = Flatten(name='disc_flatten')(x)
        result = Dense(units=1, # reinforcement style yes/no learning
                       activation='sigmoid',
                       kernel_initializer=self.weight_init,
                       name='disc_output')(x)
        
        return Model(input_layer, result)
    
    def _build_generator(self) -> Model:
        def n(part: str, idx: int) -> str:
            return f"gen_{part}_{idx:02}"
        
        input_layer = Input(shape=self.z_dim, name='gen_input')
        x = input_layer
        
        # Do a Dense product on the flattend representation, then reshape
        # it so that it looks like 
        x = Dense(np.prod(self.gen_initial_dense_layer_size), name=n('expand', 0))(x)
        if self.gen_batch_norm_momentum:
            x = BatchNormalization(momentum=self.gen_batch_norm_momentum, name=n('bnorm', 0))(x)
        x = Activation(self.gen_activation, name=n(self.gen_activation, 0))(x)
        x = Reshape(target_shape=self.gen_initial_dense_layer_size, name=n('reshape', 0))(x)
        if self.gen_dropout_rate:
            x = Dropout(self.gen_dropout_rate, name=n('dropout', 0))(x)
        
        for idx, (ups, fltrs, ksize, stride) in enumerate(zip(self.gen_upsample,
                                                              self.gen_conv_filter_count,
                                                              self.gen_conv_kernel_size,
                                                              self.gen_conv_strides
                                                             )
                                                          ):
            idx += 1  # one-indexed

            if ups != 1:
                x = UpSampling2D(size=(ups, ups), name=n(f"up_{ups}", idx))(x)
            x = Conv2D(filters=fltrs, 
                       kernel_size=ksize, 
                       strides=stride, 
                       padding='same',
                       name=n('conv', idx))(x)
            if self.gen_batch_norm_momentum:
                x = BatchNormalization(momentum=self.gen_batch_norm_momentum, name=n('bnorm', idx))(x)
            
            act = 'tanh' if idx == len(self.gen_conv_filter_count) else self.gen_activation
            x = Activation(act, name=n(self.gen_activation, idx))(x)
#             if self.gen_dropout_rate:  # Apparently this is not in the generator, but it was in the Celebfaces decoder ?!
#                 x = Dropout(self.gen_dropout_rate, name=n("dropout", idx))
            
        result = x
        return Model(input_layer, result)
    

In [ ]:
gan_builder = GanBuilder(
    input_dim = QUICK_DRAW_SHAPE,
    
    disc_conv_filter_count=[64, 64, 128, 128],
    disc_conv_kernel_size=[5, 5, 5, 5],
    disc_conv_strides=[2, 2, 2, 1],
    disc_batch_norm_momentum=None,
    disc_activation='relu',  # Note harder node compared to VAE
    disc_dropout_rate=0.4,
    disc_learning_rate=0.0006,
    
    gen_initial_dense_layer_size=(7, 7, 64),
    gen_upsample = [2, 2, 1, 1],
    gen_conv_filter_count =[128, 64, 64, 1],
    gen_conv_kernel_size=[5, 5, 5, 5],
    gen_conv_strides=[1, 1, 1, 1],
    gen_batch_norm_momentum=0.9,  # Note swap of dropout/bnorm vs disc
    gen_activation='relu',
    gen_dropout_rate=None,
    gen_learning_rate=0.0002,
    
    optimiser='rms_prop',
    weight_init=tf.keras.initializers.RandomNormal(mean=0., stddev=0.2),
    z_dim=100
)

In [ ]:
gan_builder.discriminator_model.summary()

In [ ]:
gan_builder.generator_model.summary()

In [ ]:
model = gan_builder.compiled_model

In [ ]:
model.summary()

In [ ]:
gan_builder.discriminator_model.trainable

In [ ]:
model.summary()

In [ ]:
tensor_flow_shape = tuple([len(camels)] + list(QUICK_DRAW_SHAPE))
camels_to_train = camels.reshape(tensor_flow_shape)

In [ ]:
def center_and_scale(arr: np.ndarray) -> Tuple[float, float, np.ndarray]:
    """
    Centre and scale the dataset.
    
    Return the mean and std before centering and scaling, and
    the transformed dataset.
    """
    mn = np.mean(arr)
    sd = np.std(arr)
    
    return mn, sd, (arr - mn) / sd

mn, sd, camels_to_train = center_and_scale(camels_to_train)

In [ ]:
plt.imshow(camels_to_train[12132], cmap='Greys')

In [ ]:
gan_builder.train(x_train=camels_to_train, epoch_count=1000, batch_size=1000)

In [ ]:
epochs = np.arange(0, gan_builder.epochs_completed + 1)

In [ ]:
plt.plot(epochs, gan_builder.disc_loss_real, label='D-Real')
plt.plot(epochs, gan_builder.disc_loss_fake, label='D-Fake')
plt.plot(epochs[:-1], gan_builder.gen_loss, label='G')
plt.legend()

In [ ]:
predicted_images = gan_builder.generator_model.predict(
    gan_builder._low_dim_image_sample(batch_size=64)
)
plot_camels(predicted_images)

# To release memory, in order to switch to a different notebook

First check memory with 

```
nvidia-smi
```

or, if you have installed it

```
gpustat
```

Then there are two approaches. The first uses Keras

```
from tensorflow.keras import backend as K
K.clear_session()
```

the second is

```
from numba import cuda
cuda.select_device(0)
cuda.close()
```

I suspect if I were using a managed session, instead of an interactive one, I might not have to use these hacks

In [ ]:
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

In [ ]:
!nvidia-smi